In [8]:
from dask import dataframe as dd
import dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
from dask import multiprocessing
import  numpy as np

blockSize=10e6
fileRead="Toys_and_Games.csv"
fileWrite="data/"+fileRead
parquestWrite=fileRead.split(".")[0]+str(blockSize)


cluster = LocalCluster(n_workers=2, threads_per_worker=2,memory_limit='10GB')
client = Client(cluster, asynchronous=True)
display(client)

data=dd.read_csv(fileRead, blocksize=blockSize,names=["user", "item", "rating", "timestamp"])
print(data.head())
itemSize=data.item.count().compute()
print(itemSize)
%time itemLabel=data['item'].unique().compute().factorize()
%time userLabel=data['user'].unique().compute().factorize()

print(itemLabel)

print(userLabel)

/home/summer/.local/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41333 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:36019 Dashboard: http://127.0.0.1:41333/status,Cluster Workers: 2 Cores: 4 Memory: 18.63 GiB


         user            item  rating   timestamp
0  0020232233  A1IDMI31WEANAF     2.0  1474502400
1  0020232233   A4BCEVVZ4Y3V3     1.0  1474156800
2  0020232233  A2EZ9PY1IHHBX0     3.0  1473638400
3  0020232233  A139PXTTC2LGHZ     5.0  1488412800
4  0020232233  A3IB33V29XIL8O     1.0  1486512000
8201231
CPU times: user 4.43 s, sys: 563 ms, total: 5 s
Wall time: 22.9 s
CPU times: user 1.15 s, sys: 290 ms, total: 1.44 s
Wall time: 6.88 s
(array([      0,       1,       2, ..., 4204991, 4204992, 4204993]), Index(['A1IDMI31WEANAF', 'A4BCEVVZ4Y3V3', 'A2EZ9PY1IHHBX0', 'A139PXTTC2LGHZ',
       'A3IB33V29XIL8O', 'A1J86V48S4KRJE', 'A14J12PRBLGHF4', 'A2UKOWP9ICU416',
       'A2ONKKDETRWT79', 'AK9GN9KZZNTEP',
       ...
       'A3FFQ7G5V45WJ5', 'AA61YB68DLK65', 'A29DE6IOYT0MM9', 'A32OT82VP12NVC',
       'A2L759QVX3KTL6', 'A1SU8WSUR49A1E', 'A2O43XSWKAE1R5', 'A1W75PEIIL0IFM',
       'A3OCDEVI6FGUWU', 'A3N28JAZYS4L9O'],
      dtype='object', length=4204994))
(array([     0,      1,      2, ..., 6

In [9]:
%%time
filehandle = open(fileRead, 'r')
f = open(fileWrite, "a")
i=0
while True:
        # read a single line
        line = filehandle.readline()
        if not line:
            break
        lines=line.split(",")
        lines[0]=str(userLabel[1].get_loc(lines[0]))
        lines[1]=str(itemLabel[1].get_loc(lines[1]))
        lines.pop(-1)
        f.write(",".join(lines)+"\n")
        i+=1
        if(i%10000000)==0:
            print(i/itemSize)
f.close()
filehandle.close()

KeyError: '0020232233'

In [6]:
%%time
df = dd.read_csv(fileWrite, blocksize=blockSize,names=["user", "item", "rating"])
print(df.head())
df.to_parquet(parquestWrite)

   item  user  rating
0     0     0     5.0
1     0     1     5.0
2     0     2     5.0
3     0     3     4.0
4     0     4     5.0
CPU times: user 603 ms, sys: 67.3 ms, total: 670 ms
Wall time: 5.82 s


In [5]:
'''from dask.distributed import Variable
futureUser = client.scatter(userLabel[1], broadcast=False)
userLabelVar = Variable(name="userLabel")
userLabelVar.set(futureUser)
futureItem = client.scatter(itemLabel[1], broadcast=False)
itemLabelVar = Variable(name="itemLabel")
itemLabelVar.set(futureItem)'''

'from dask.distributed import Variable\nfutureUser = client.scatter(userLabel[1], broadcast=False)\nuserLabelVar = Variable(name="userLabel")\nuserLabelVar.set(futureUser)\nfutureItem = client.scatter(itemLabel[1], broadcast=False)\nitemLabelVar = Variable(name="itemLabel")\nitemLabelVar.set(futureItem)'

In [7]:
from dask import delayed
import pandas as pd
def mapRow(row):
    row.user=(userLabel[1].get_loc(row.user))
    row.item=(itemLabel[1].get_loc(row.item))
    return row

def mapPartition(partition):
    partition["user"]=df["user"].map(userLabelVar.get().get_loc)
    partition["item"]=df["item"].map(itemLabelVar.get().get_loc)
    return partition
    

In [7]:
#data.map_partitions(lambda part: delayed(mapPartition)(part,userLabel,itemLabel),meta=object).compute().head()